In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import models, layers, activations
from tensorflow.keras.layers import Conv2D, TimeDistributed, BatchNormalization, MaxPooling2D, Flatten, Bidirectional, Dense,Dropout,Flatten
import coremltools as ct
from coremltools.proto import NeuralNetwork_pb2, FeatureTypes_pb2

In [5]:
x_data = np.load('lip_x_data.npy') # x_save.npy
y_data = np.load('lip_y_data.npy') # y_data.npy
y_data = keras.utils.to_categorical(y_data,11)
print(x_data.shape, y_data.shape)

(3000, 27, 64, 64, 3) (3000, 11)


In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(x_data, y_data, test_size = 0.2, shuffle = True, stratify=y_data)
print("x_train: {}, X_test: {}".format(X_train.shape, X_test.shape))
print("Y_train: {}, Y_test: {}".format(Y_train.shape, Y_test.shape))

x_train: (2400, 27, 64, 64, 3), X_test: (600, 27, 64, 64, 3)
Y_train: (2400, 11), Y_test: (600, 11)


In [8]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import Input


# this could also be the output a different Keras model or layer
#input_tensor = Input(shape=(224, 224, 3))

#model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=True)

class MobileNet_LSTM(tf.keras.Model):
    def __init__(self, num_classes):
        super(MobileNet_LSTM, self).__init__()
        #self.BZ = input_shape[0]
        #self.frame = input_shape[1]

        self.mobile = TimeDistributed(MobileNet(weights='imagenet', include_top=False))
        #self.vgg.trainable = False

        self.bilstm = layers.Bidirectional(layers.LSTM(256), merge_mode='concat')
        self.dense = layers.Dense(num_classes, activation='softmax')
        self.max = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))
        self.dropout = layers.Dropout(0.2)
        self.flat= TimeDistributed(Flatten())

    def call(self, x):
        x = self.mobile(x)
        x = self.max(x)
        x = self.dropout(x)      
        x = self.flat(x)
        x = self.dropout(x)
        x = self.bilstm(x)
        return self.dense(x)

In [9]:
model = MobileNet_LSTM(num_classes = 11)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [10]:
model.load_weights('mobileNet_lip')

In [11]:
# Y_test의 index 1~10 까지 60개씩 존재 : 총 600개
predictions = model.predict(X_test)

In [33]:
# Convert the model
inputs = ct.ImageType(shape=(1, 27, 64, 64, 3))
class_labels = Y_test[0].shape
# class_labels = 11
classfiler_config = ct.ClassifierConfig(class_labels=class_labels)

mlmodel = ct.convert(
    model, inputs=[inputs], classfiler_config=classfiler_config,)

InvalidArgumentError: Attempting to add a duplicate function with name: while_cond_53617 where the previous and current definitions differ. Previous definiton: signature {
  name: "while_cond_53617"
  input_arg {
    name: "while_while_loop_counter"
    type: DT_INT32
  }
  input_arg {
    name: "while_while_maximum_iterations"
    type: DT_INT32
  }
  input_arg {
    name: "while_placeholder"
    type: DT_INT32
  }
  input_arg {
    name: "while_placeholder_1"
    type: DT_VARIANT
  }
  input_arg {
    name: "while_placeholder_2"
    type: DT_FLOAT
  }
  input_arg {
    name: "while_placeholder_3"
    type: DT_FLOAT
  }
  input_arg {
    name: "while_less_strided_slice"
    type: DT_INT32
  }
  input_arg {
    name: "while_while_cond_53617___redundant_placeholder0"
    type: DT_VARIANT
  }
  input_arg {
    name: "while_while_cond_53617___redundant_placeholder1"
    type: DT_FLOAT
  }
  input_arg {
    name: "while_while_cond_53617___redundant_placeholder2"
    type: DT_FLOAT
  }
  in

In [26]:
print(Y_test[0:10])

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
